In [13]:
from datetime import datetime
import pandas as pd
from meteostat import Point, Daily, Hourly

# Parameters
start = datetime(1995, 10, 1)
end   = datetime(2025, 6, 1)
location = Point(35.4333, -82.0333, 660)

# Fetch daily data directly - this is more efficient
df_met = Daily(location, start, end).fetch()

# Select and rename columns for clarity
df_met = df_met[['tavg', 'prcp', 'wspd', 'pres']].round(2)
df_met.index.name = 'datetime'

# add relative humidity to daily data
hourly_met = Hourly(location, start, end).fetch()
df_met['rhum'] = hourly_met['rhum'].resample('D').mean().round(2)

print("Weather data columns:", df_met.columns.tolist())
print("Missing proportion per column:\n", df_met.isna().mean())

# Interpolate any missing values using time-based interpolation
df_met = df_met.interpolate(method='time')

print("\nFinal weather data snapshot:")
print(df_met.head())
print(f"Daily weather data shape: {df_met.shape}")

# --- Loading and Combining ---

print("\nLoading and combining datasets...")

# Load water level data and set the index
water_df = pd.read_csv("dataset.csv", parse_dates=['datetime']).set_index('datetime')

print(f"Weather dataset shape: {df_met.shape}")
print(f"Water level dataset shape: {water_df.shape}")

# --- FIX THE TIMEZONE MISMATCH ---
# The water_df index is timezone-aware (UTC), while df_met is naive.
# We make the water_df index naive to allow for the join.
water_df.index = water_df.index.tz_localize(None)
# ---------------------------------

# Now that both are naive, we can join them.
# An inner join ensures we only keep dates that exist in both datasets.
combined_df = df_met.join(water_df, how='inner')

# As a final check, drop any rows that might have NaNs after the join
combined_df.dropna(inplace=True)

print(f"\nCombined dataset shape: {combined_df.shape}")
print("Combined data sample:")
print(combined_df.head())

# Check for any remaining missing values
print(f"\nMissing values in combined dataset:")
print(combined_df.isnull().sum())

# Make the 'stage_m' and 'discharge_cms' columns the first two columns if they exist
first_cols = []
for col in ['stage_m', 'discharge_cms']:
    if col in combined_df.columns:
        first_cols.append(col)
remaining_cols = [col for col in combined_df.columns if col not in first_cols]
combined_df = combined_df[first_cols + remaining_cols]

# Save the final combined dataset
combined_df.to_csv("combined_dataset.csv")
print(f"\nCombined dataset saved as 'combined_dataset.csv'")
print(f"Final dataset has {len(combined_df)} rows and {len(combined_df.columns)} columns")
print(f"Columns: {combined_df.columns.tolist()}")


Weather data columns: ['tavg', 'prcp', 'wspd', 'pres', 'rhum']
Missing proportion per column:
 tavg    0.000369
prcp    0.000000
wspd    0.000000
pres    0.005444
rhum    0.000000
dtype: float64

Final weather data snapshot:
            tavg  prcp  wspd    pres   rhum
datetime                                   
1995-10-01  17.9   0.0   7.2  1020.5  82.83
1995-10-02  18.0   0.0   7.2  1018.4  83.46
1995-10-03  18.0  13.2   8.6  1014.4  93.92
1995-10-04  16.9  84.3   4.7  1010.7  98.79
1995-10-05  20.4  49.0  26.6  1001.8  91.67
Daily weather data shape: (10837, 5)

Loading and combining datasets...
Weather dataset shape: (10837, 5)
Water level dataset shape: (10815, 2)

Combined dataset shape: (10815, 7)
Combined data sample:
            tavg  prcp  wspd    pres   rhum  stage_m  discharge_cms
datetime                                                           
1995-10-01  17.9   0.0   7.2  1020.5  82.83    1.960         17.840
1995-10-02  18.0   0.0   7.2  1018.4  83.46    1.932         